In [0]:
silver_adls = "abfss://silver-layer@medallionapistorage.dfs.core.windows.net/"
gold_adls = "abfss://gold-layer@medallionapistorage.dfs.core.windows.net/"

In [0]:
import datetime 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# Criando uma SparkSession
spark = SparkSession.builder.appName("Open Brewery").getOrCreate()


In [0]:
#Cria dataframe spark com os dados persistidos no formato delta e particionados na camada silver
df = spark.read.format("delta").load(f"{silver_adls}open_breweries_delta")
#display(df)

In [0]:
#Agrega os valores por tipo de cervejaria, cidade, estado e país
df_gold = (
    df.groupBy("country", "state", "city", "brewery_type")
    .count().alias("breweries_qty")
    .withColumnRenamed("count", "breweries_qty")
    .orderBy("country", "state", "city", "brewery_type")
    )

#display(df_gold)

#Salva o dataframe com os dados agregados na camada gold, no formato delta
#Indica o caminho para persistir os dados
delta_path = f"{gold_adls}open_breweries_agg"

#Salva os dados no formato delta
(
    df_gold
    .write
    .mode("overwrite")
    .format("delta")
    .partitionBy("country", "state")
    .save(delta_path)
)

print('Dados agregados salvos na camada gold')

# Cria tabela delta para facilitar visualização e consulta dos dados
(
    df_gold
    .write 
    .format("delta") 
    .mode("overwrite") 
    .option("path", delta_path) 
    .saveAsTable("brewery.default.open_brewery_agg")
)
